# 01 - Data Wrangling 🧹

## Problem

How can we generate short descriptions including a list of popular beer styles from brewery reviews with natural language processing (NLP)? 

NOTE: This is similar to how Google Maps provides short blurbs for businesses such as “From scratch, Northern Italian dining.” 

For example, "Spacious warehouse brewery with daily food trucks. Allows dogs. Features IPAs, Hazy IPAs, and high-gravity stouts."

## Notebook Objectives

* Clean the dataset and address issues like missing values and duplicate values. 
* Apply data wrangling techniques, as laid out in the DSM of; Data Collection, Data Organization, Data Definition, Data Cleaning.

## Method

This notebook will go through scraping the Beer Advocate site for breweries and reviews. 

1. Scrape **[Beer Advocate](https://www.beeradvocate.com)** Places for cities (86 cities)
2. Loop through the cities and store Breweries (~5000 breweries)
3. Loop through breweries and store Reviews

All data will be stored in a SQLite DB.

## DB Models

* Cities
    * id (Integer)
    * name (String)
    * url (String)
    * ba_city_id (Integer)
* Breweries
    * id (Integer)
    * name (String)
    * street (String)
    * city_id (Integer)
    * url (String)
    * ba_brewery_id (Integer)
* Reviews
    * id (Integer)
    * text (Text)
    * date (Datetime)
    * rating (Float)
    * ba_brewery_id (Integer)

In [2]:
import re
import requests
import numpy as np
import pandas as pd
import time
from datetime import datetime

# Progress bar
from tqdm import tqdm, trange

# Beautiful Soup for HTML parsing
from bs4 import BeautifulSoup

# SQLAlchemy ORM for SQLite
from sqlalchemy import create_engine, desc, Column, Date, Integer, Float, String, UnicodeText, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [4]:
# Build the DB models declaratively (i.e., Pythonic way)
Base = declarative_base()

class City(Base):
    __tablename__ = 'cities'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    url = Column(String)
    ba_city_id = Column(Integer)
    
class Brewery(Base):
    # FIXME: Rename to 'breweries'
    __tablename__ = 'brewery'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    street = Column(String)
    city_id = Column(Integer)
    url = Column(String)
    ba_brewery_id = Column(Integer)
    
class Review(Base):
    __tablename__ = 'reviews'
    id = Column(Integer, primary_key=True)
    rating = Column(Float)
    text = Column(UnicodeText)
    date = Column(Date)
    ba_brewery_id = Column(Integer)
    
engine = create_engine(f"sqlite:///../data/processed/brewery_reviews.db")

# Set up the Session
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

# Ensure the Tables are created
Base.metadata.create_all(engine)

In [6]:
# Inspect DB counts
num_cities = session.query(City).count()
num_breweries = session.query(Brewery).count()
num_reviews = session.query(Review).count()
print(f"Number of cities = {num_cities}")
print(f"Number of breweries = {num_breweries}")
print(f"Number of reviews = {num_reviews}")

Number of cities = 86
Number of breweries = 4984
Number of reviews = 45417


In [7]:
# Setup variables
ba_domain = "https://www.beeradvocate.com"
places_url = "https://www.beeradvocate.com/place/"
request_headers = {'User-agent': 'Springboard Capstone Project Research Tool by Chris Mears <chris.mears@gmail.com>'}

In [8]:
# Get the page and parse
places_page = requests.get(places_url, headers=request_headers)
places_soup = BeautifulSoup(places_page.text)

In [9]:
# Select all of the brewery items
# <a href="/place/city/73/">Anchorage</a>
cities = places_soup.find_all(href=re.compile("\/place\/city\/"))
print(f"Number of Cities: {len(cities)}")

Number of Cities: 86


In [10]:
# Clean-up: Delete all rows in City
# session.query(City).delete()

In [22]:
# Add cities to DB
for city in cities:
    name = city.string
    url = f"{ba_domain}{city['href']}"
    ba_city_id = re.match(r"\/place\/city\/(\d+)\/", city['href']).group(1)
    
    # Skip if city already exists
    if session.query(City).filter(City.ba_city_id==ba_city_id).first() is None:
        new_city = City(name=name, url=url, ba_city_id=ba_city_id)
        session.add(new_city)

# Write to DB
session.commit()

In [23]:
# Inspect
for city in session.query(City).limit(10):
    print(city.id, city.name, city.url, city.ba_city_id)

1 Birmingham https://www.beeradvocate.com/place/city/83/ 83
2 Anchorage https://www.beeradvocate.com/place/city/73/ 73
3 Phoenix https://www.beeradvocate.com/place/city/29/ 29
4 Los Angeles https://www.beeradvocate.com/place/city/9/ 9
5 San Diego https://www.beeradvocate.com/place/city/28/ 28
6 San Francisco https://www.beeradvocate.com/place/city/17/ 17
7 Boulder https://www.beeradvocate.com/place/city/79/ 79
8 Denver https://www.beeradvocate.com/place/city/7/ 7
9 Fort Collins https://www.beeradvocate.com/place/city/53/ 53
10 Hartford https://www.beeradvocate.com/place/city/38/ 38


In [26]:
# Loop through City pages scraping Breweries (86 requests)
cities = session.query(City).all()

for city in tqdm(cities):
    # Get city page and parse
    city_page = requests.get(city.url, headers=request_headers)
    city_soup = BeautifulSoup(city_page.text)    
    
    # Find breweries by URL regex pattern
    breweries = city_soup.find(id="ba-content").find_all(href=re.compile("\/beer\/profile\/"))
    
    # Loops through breweries, scrape, and add to DB
    for brewery in breweries:
        ba_brewery_id = re.match(r"\/beer\/profile\/(\d+)\/", brewery['href']).group(1)
        url = f"{ba_domain}{brewery['href']}"

        try:
            street_scrape = brewery.parent.find(class_='muted').get_text()
            street = street_scrape.replace(' - ', '').replace(f", {city.name}", "")
        except:
            street = None
        
        # Skip if it already exists
        if session.query(Brewery).filter(Brewery.ba_brewery_id==ba_brewery_id).first() is None:
            new_brewery = Brewery(name=brewery.string, street=street, url=url, city_id=city.id, ba_brewery_id=ba_brewery_id)
            session.add(new_brewery)

        # Write to DB
        session.commit()
    
    # Sleep for 1 sec (ethical scraping)
    time.sleep(1)

100%|██████████| 86/86 [02:30<00:00,  1.75s/it]


In [27]:
# Inspect scraped Breweries
breweries = session.query(Brewery).order_by(desc(Brewery.id)).limit(25)
for brewery in breweries:
    print(brewery.id, brewery.name, brewery.street, brewery.url, brewery.ba_brewery_id)

4984 PGW Produkter AB Cementvägen 34, Haninge https://www.beeradvocate.com/beer/profile/7219/?view=ratings 7219
4983 Systembolaget - Passagen Regeringsgatan 44 https://www.beeradvocate.com/beer/profile/10916/?view=ratings 10916
4982 Oliver Twist Repslagargatan 6 https://www.beeradvocate.com/beer/profile/5299/?view=ratings 5299
4981 Belgobaren Bryggargatan 12 https://www.beeradvocate.com/beer/profile/10949/?view=ratings 10949
4980 Man in the Moon Tegnérgatan 2 https://www.beeradvocate.com/beer/profile/21842/?view=ratings 21842
4979 BrewDog Stockholm Sankt Eriksgatan 56 https://www.beeradvocate.com/beer/profile/31801/?view=ratings 31801
4978 Duvel Café Vasagatan 50 https://www.beeradvocate.com/beer/profile/10522/?view=ratings 10522
4977 Ardbeg Embassy Västerlångatan 68 https://www.beeradvocate.com/beer/profile/10906/?view=ratings 10906
4976 Omnipollos Hatt Hökens gata 3 https://www.beeradvocate.com/beer/profile/41828/?view=ratings 41828
4975 Akkurat Hornsgatan 18 https://www.beeradvocate

In [28]:
# Test things out by reading the 2nd brewery reviews page
brewery = session.query(Brewery)[1]
brewery_url = f"{ba_domain}/beer/profile/{brewery.ba_brewery_id}/?view=ratings"
brewery_page = requests.get(brewery_url, headers=request_headers)
brewery_soup = BeautifulSoup(brewery_page.text)
reviews = brewery_soup.find_all(id="rating_fullview_content_2")
print(f"Number of Reviews: {len(reviews)}")

In [65]:
# Define regex pattern for user profile URLs
def user_profile(href):
    return href and re.compile("\/beer\/profile\/").search(href)

In [ ]:
# Scrape Brewery Reviews
# NOTE: Scraping 500 breweries takes ~15 minutes

# Set variables
offset = 0
n = 100
breweries = session.query(Brewery).order_by(Brewery.id).offset(offset).limit(n)

# Loop through breweries
for i in trange(n):
    # Request brewery page and parse
    brewery = breweries[i]
    brewery_url = f"{ba_domain}/beer/profile/{brewery.ba_brewery_id}/?view=ratings"
    brewery_page = requests.get(brewery_url, headers=request_headers)
    brewery_soup = BeautifulSoup(brewery_page.text)

    # Get the reviews
    reviews = brewery_soup.find_all(id="rating_fullview_content_2")
    
    for review in reviews:
        # Get the rating
        rating = float(review.find(class_="BAscore_norm").text)

        try:
            # Parse the date (linked via user's profile URL)
            date_string = datetime.strptime(review.find(href=user_profile).text, "%b %d, %Y")
        except:
            date_string = datetime.now()

        # Parse review text         
        text = []
        for sibling in review.find_all(class_="muted")[1].next_siblings:
            # Review text isn't wrapped in tags :shrug:
            if sibling.name == None:
                text.append(sibling)
        review = " ".join(text)

        # DEBUG
        # print(f"Rating: {rating}")
        # print(f"Date: {date_string}")
        # print(f"Review: {review}")
        # print(f"BA Brewery ID: {brewery.ba_brewery_id}")

        # Add Review to DB
        new_review = Review(rating=rating, date=date_string, text=review, ba_brewery_id=brewery.ba_brewery_id)
        session.add(new_review)

    # Save Brewery Reviews
    session.commit()
    
    # Sleep for 1 sec (ethical scraping)
    time.sleep(1)

In [80]:
# Check how many reviews we have
len(session.query(Review).all())

45417

In [79]:
# Inspect the last 10 reviews
for review in session.query(Review).order_by(desc(Review.id)).limit(10):
    print(review.id, review.rating, review.text, review.date, review.ba_brewery_id)

45417 4.3 Had the privilege to have a few drinks here while visiting Amsterdam in September 2011. 2012-01-13 25734
45416 3.58 An American beer bar in Europe was intriguing enough to have to pay it a visit. It kind of had the vibe of a Belgian beer bar in America. Especially when it came to price, in Holland & Belgium their beers are very cheap compared to what we see in the US. Here the US beer prices were comparable to Belgian prices in the US.  2011-02-13 21770
45415 4.58 This is a must stop for anyone visiting Amsterdam or locals who haven't been exposed to American craft beer. BeerTemple is small in size, but absolutely huge in terms of selecion. In fact, they have a better American craft selection than anywhere here in Nashville. 2011-02-28 21770
45414 4.14 Beer temple was my final stop during my St Patrick's day Amsterdam beer bar crawl. It is best described as an American beer bar smack in the middle of Amsterdam. Founders, Jolly Pumpkin, Great Divide, Anchor, Left Hand, Flying 

In [55]:
# FOR ISSUES THAT NEED ROLLBACK
# session.rollback()

## Lessons Learned

* A script would likely be more suitable than a notebook.
* SQLite is useful for checking for existing records, but the same could have probably been done with Pandas and exporting to a CSV. 